In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

seed = 42
rng = np.random.default_rng(seed)
N = 50
MAX_RANGE = 50
nodes = list(range(N))
print(nodes)
node_locs = rng.normal(scale=MAX_RANGE / 2 / 3, size=(N, 2))
node_locs[0, :] = 0  # The first node is the depot node (at 0,0)
print(node_locs)
edges = list(edge for edge in itertools.product(nodes, nodes) if edge[0] != edge[1])
print(edges)
distances = [
    np.linalg.norm(node_locs[pt1, :] - node_locs[pt2, :]) for pt1, pt2 in edges
]
distances_dict = {edge: distance for edge, distance in zip(edges, distances)}
print(distances)

plt.plot(node_locs[0, 0], node_locs[0, 1], "or")
plt.plot(node_locs[1:, 0], node_locs[1:, 1], "x")


In [ ]:
import gurobipy as gp
import matplotlib.pyplot as plt
import networkx as nx

model = gp.Model()
x = model.addVars(edges, obj=distances_dict, vtype=gp.GRB.BINARY, name="x")
r = model.addVars(nodes, vtype=gp.GRB.CONTINUOUS, name="range")

# Conserve flow at depot node
model.addConstr(x.sum(nodes[0], "*") == x.sum("*", nodes[0]))
# Ensure at least one tour goes through the depot
model.addConstr(x.sum(nodes[0], "*") >= 1)
# Set the residual range at the depot to the max
model.addConstr(r[nodes[0]] == MAX_RANGE)

# Only one exit from each non-depot node
model.addConstrs(x.sum(node, "*") == 1 for node in nodes[1:])
# Only one entrance to each non-depot node
model.addConstrs(x.sum("*", node) == 1 for node in nodes[1:])

# Ensure residual range is always enough to complete route
model.addConstrs(r[node] >= distances_dict[node, nodes[0]] for node in nodes[1:])
# Ensure residual range doesn't go above max (may be redundant)
model.addConstrs(r[node] <= MAX_RANGE for node in nodes)
# Constrain the residual range to be propagated through a subtour
# Create equality constraint that only holds when x[i,j] is True (1)
# Don't apply to returning to the depot (j == nodes[0])
model.addConstrs(
    r[j] >= r[i] - distances_dict[i, j] - MAX_RANGE * (1 - x[i, j])
    for i, j in itertools.product(nodes, nodes[1:])
    if i != j
)
model.addConstrs(
    r[j] <= r[i] - distances_dict[i, j] + MAX_RANGE * (1 - x[i, j])
    for i, j in itertools.product(nodes, nodes[1:])
    if i != j
)


# def eliminate_bad_subtours(model, where, x):
#     if where == gp.GRB.Callback.MIPSOL:
#         vals = model.cbGetSolution(x)
#         selected = gp.tuplelist((i, j) for i, j in x.keys() if vals[i, j] > 0.5)
#         # print(selected)
#         subtours = extract_subtours(selected)
#         # print(subtours)
#         for subtour in subtours:
#             if not depot_in_subtour(subtour):
#                 disallow_subtour(model, subtour, x)
#             if total_distance(subtour, x) > MAX_RANGE:
#                 disallow_subtour(model, subtour, x)


def heuristic_solution_cb(model, where, x, nodes, distances):
    if where == gp.GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(x)
        selected = gp.tuplelist((i, j) for i, j in x.keys() if vals[i, j] > 0.5)
        # print(selected)
        # subtours = extract_subtours(selected)
        graph = nx.DiGraph(selected)
        did_improve, improved_solution = improve_solution(graph, x, nodes, distances)
        if did_improve:
            model.cbSetSolution(x, improved_solution)


def extract_subtours(selected):
    graph = nx.DiGraph(selected)
    subtours = []
    visited = set()
    visited.add(nodes[0])
    for next_node in graph.successors(nodes[0]):
        subtour = [nodes[0]]
        while next_node != nodes[0]:
            subtour.append(next_node)
            visited.add(next_node)
            next_nodes = list(graph.successors(next_node))
            assert len(next_nodes) == 1
            next_node = next_nodes[0]
        subtours.append(subtour)

    for node in nodes[1:]:
        if node in visited:
            continue
        subtour = [node]
        next_nodes = list(graph.successors(node))
        assert len(next_nodes) == 1
        next_node = next_nodes[0]
        while next_node != node:
            subtour.append(next_node)
            visited.add(next_node)
            next_nodes = list(graph.successors(next_node))
            assert len(next_nodes) == 1
            next_node = next_nodes[0]
        subtours.append(subtour)
    return subtours


def improve_solution(subtour_graph, x, nodes, distances):
    improved_solution = False
    for node in subtour_graph.nodes:
        if node == nodes[0]:
            continue
        prev_ = next(subtour_graph.predecessors(node))
        next_ = next(subtour_graph.successors(node))
        improvements = []
        for other_node in subtour_graph.nodes:
            if node == other_node:
                continue
            other_next = next(subtour_graph.successors(other_node))
            curr_cost = (
                distances[prev_, node]
                + distances[node, next_]
                + distances[other_node, other_next]
            )
            try:
                swapped_cost = (
                    distances[other_node, node]
                    + distances[node, other_next]
                    + distances[prev_, next_]
                )
            except KeyError:
                continue
            cost_improvement = curr_cost - swapped_cost
            improvements.append((cost_improvement, (other_node, other_next)))
        if not improvements:
            continue
        max_improvement = max(improvements, key=lambda x: x[0])
        if max_improvement[0] > 0:
            improved_solution = True
            subtour_graph.remove_edges_from(
                ((prev_, node), (node, next_), max_improvement[1])
            )
            new_prev, new_next = max_improvement[1]
            subtour_graph.add_edges_from(
                ((prev_, next_), (new_prev, node), (node, new_next))
            )
    solution = {}
    for i, j in x:
        if subtour_graph.has_edge(i, j):
            solution[i, j] = 1
        else:
            solution[i, j] = 0
    if improved_solution:
        print("Improved the solution!")
    return improved_solution, solution


# def total_distance(subtour, x):
#     distance = 0.0
#     for edge in subtour:
#         distance += x[edge].getAttr("obj")
#     return distance


# def depot_in_subtour(subtour):
#     return any(i == 0 for i, _ in subtour) and any(j == 0 for _, j in subtour)


# def disallow_subtour(model, subtour, x):
#     # print(f"Dissallowing subtour: {subtour}")
#     model.cbLazy(gp.quicksum(x[i, j] for i, j in subtour) <= len(subtour) - 1)


model.Params.LazyConstraints = 1
# model.Params.OutputFlag = 0
model.optimize(
    lambda model, where: heuristic_solution_cb(
        model, where, x=x, nodes=nodes, distances=distances_dict
    )
)

vals = model.getAttr("x", x)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
print(selected)
tours = extract_subtours(selected)
print(tours)
plt.plot(node_locs[0, 0], node_locs[0, 1], "or")
plt.plot(node_locs[1:, 0], node_locs[1:, 1], "xb")
for tour in tours:
    pts = np.zeros((len(tour) + 1, 2))
    for idx, node in enumerate(tour):
        pts[idx] = node_locs[node]
    pts[-1] = node_locs[0]
    plt.plot(pts[:, 0], pts[:, 1])
